# Tratamento do Texto

In [1]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np
import re

In [2]:
with open('StackOverflow_dataset.pickle', 'rb') as data:
    df = pickle.load(data)

In [3]:
df.head()

,id,framework,html,sample,title,state,created_at,body,labels,Body_length
0,4,Android,https://github.com/googlesamples/io2014-codela...,googlesamples/io2014-codelabs,The main README file is not about Google I/O n...,closed,2014-07-12 00:11:32,I'm not sure what kind of information we shoul...,bug,108
1,58,Android,https://github.com/googlesamples/google-servic...,googlesamples/google-services,Android Google Sign-in: When Google account po...,open,2019-11-14 11:45:20,### Step 1: Are you in the right place?\n\n *...,bug,1250
2,65,Android,https://github.com/googlesamples/google-servic...,googlesamples/google-services,Cancelling google sign in on ios gives wrong e...,closed,2019-10-02 09:40:13,### Describe your environment\n\n * Device: i...,bug,1281
3,111,Android,https://github.com/googlesamples/google-servic...,googlesamples/google-services,Canceling a Sign in leads to GoogleSignInStatu...,closed,2018-07-16 13:53:20,### Step 1: Are you in the right place?\n\nA b...,bug,2169
4,135,Android,https://github.com/googlesamples/google-servic...,googlesamples/google-services,SignIn Dialog from GoogleSignInClient.getSignI...,open,2017-12-14 18:35:01,When running the sample for SignInClient every...,bug,1056


In [4]:
#df.loc[1]['body']
df['body']=df['title']+df['body']

In [5]:
df['body']

0       The main README file is not about Google I/O n...
1       Android Google Sign-in: When Google account po...
2       Cancelling google sign in on ios gives wrong e...
3       Canceling a Sign in leads to GoogleSignInStatu...
4       SignIn Dialog from GoogleSignInClient.getSignI...
                              ...                        
2912    scenario 3 - protect and call web api on azure...
2913    OpenLayers and Azure MapsVerify if Azure Maps ...
2914    Azure Arc support**Is your feature request rel...
2915    Azure Arc support**Is your feature request rel...
2916    detectEval() violates CSP when running in a Ch...
Name: body, Length: 2917, dtype: object

## Limpar o Texto

In [6]:
df['Content_Parsed_1'] = df['body'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("*", "")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("<p>", "")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("</p>", "")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("/", "")
# df['Content_Parsed_1'] = df['Content_Parsed_1'].replace(regex=r'\<.*>', value='')



<ipython-input-6-b7515069a38e>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("*", "")


In [7]:
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')

### Maiúsculo para minúsculo

In [8]:
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()

### Retirando pontos

In [9]:
punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')

<ipython-input-9-cdceb98d92a9>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')


### Removendo pronome de posse

In [10]:
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")

### Stemming e Lemmatization

Desde que aretirada de caracteres feita anteriormente, pode produzir palavras que não existem, usei o processo de lemmatization. 

In [11]:
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

------------------------------------------------------------


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sabia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sabia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
wordnet_lemmatizer = WordNetLemmatizer()

In [13]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    lemmatized_list = []
    
    text = df.loc[row]['Content_Parsed_4']
    text_words = str(text).split(" ")

    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    lemmatized_text = " ".join(lemmatized_list)
    
    lemmatized_text_list.append(lemmatized_text)

In [14]:
df['Content_Parsed_5'] = lemmatized_text_list

Pronomes

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sabia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
stop_words = list(stopwords.words('english'))

In [17]:
df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

<ipython-input-17-3b7196a1b53b>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')


In [18]:
list_columns = ["labels","body", "Content_Parsed_6"] #pegar o titulo 
df = df[list_columns]

df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})

In [19]:
df.head()

,labels,body,Content_Parsed
0,bug,The main README file is not about Google I/O n...,main readme file google io code labsi' s...
1,bug,Android Google Sign-in: When Google account po...,android google sign- google account popup se...
2,bug,Cancelling google sign in on ios gives wrong e...,cancel google sign ios give wrong error### d...
3,bug,Canceling a Sign in leads to GoogleSignInStatu...,cancel sign lead googlesigninstatuscodeserr...
4,bug,SignIn Dialog from GoogleSignInClient.getSignI...,signin dialog googlesigninclientgetsignininte...


## Mapeando as categorias

In [20]:
category_codes = {
    'bug': 0,
    'enhancement': 1,
    'question': 2,
}
#categorias vou por aqui

In [21]:
df['Category_Code'] = df['labels']
df = df.replace({'Category_Code':category_codes})

## Divisão de treino e teste

In [22]:
X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'], 
                                                    df['Category_Code'], 
                                                    test_size=0.30, 
                                                    random_state=10)

In [23]:
y_train

2577    0
1054    2
1573    0
1548    0
1717    0
       ..
2009    2
1180    0
1344    2
527     0
1289    2
Name: Category_Code, Length: 2041, dtype: int64

## Representação do texto

In [24]:
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [25]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(2041, 300)
(876, 300)


In [26]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' labels:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")


# 'bug' labels:
  . Most correlated unigrams:
. fix
. fail
. reproduce
. chrome
. support
  . Most correlated bigrams:
. step reproduce
. expect behavior

# 'enhancement' labels:
  . Most correlated unigrams:
. would
. unity
. add
. error
. support
  . Most correlated bigrams:
. image httpsuser
. feature request

# 'question' labels:
  . Most correlated unigrams:
. resolver
. get
. unity
. token
. login
  . Most correlated bigrams:
. google assistant
. web app



In [27]:
# X_train
with open('Pickles/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open('Pickles/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open('Pickles/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open('Pickles/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open('Pickles/df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
# features_train
with open('Pickles/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('Pickles/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('Pickles/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('Pickles/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open('Pickles/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)